In [153]:
import pandas as pd
import geopandas as gpd
import dask.dataframe as dd

# Colocation Maps

This dataset helps us identify where people colocated for every week. 

In [154]:
aoi = ["NER", "NGA", "MLI", "BFA", "TCD", "DZA", "BEN", "LBY"]

In [155]:
import glob

colocation = pd.DataFrame()
colocation_ddf = dd.from_pandas(colocation, npartitions=1)

for file in glob.glob("../../data/movement/meta/colocation_maps/*.csv"):
    df = dd.read_csv(file)
    df = df[df["country"].isin(aoi)]
    colocation_ddf = dd.concat([colocation_ddf, df])

In [156]:
colocation_ddf = colocation_ddf.repartition(npartitions=1)

In [157]:
niger_adm0 = gpd.read_file(
    "../../data/shapefiles/ner_adm_ignn_20230720_ab_shp/NER_admbnda_adm0_IGNN_20230720.shp"
)
niger_adm1 = gpd.read_file(
    "../../data/shapefiles/ner_adm_ignn_20230720_ab_shp/NER_admbnda_adm1_IGNN_20230720.shp"
)
niger_adm2 = gpd.read_file(
    "../../data/shapefiles/ner_adm_ignn_20230720_ab_shp/NER_admbnda_adm2_IGNN_20230720.shp"
)

In [158]:
niger_gadm = gpd.read_file("../../data/shapefiles/gadm/gadm41_NER_2.json")

In [159]:
libya = gpd.read_file(
    "../../data/shapefiles/libya/lby_admbnda_adm0_unosat_lbsc_20180507.shp"
)
nigeria = gpd.read_file(
    "../../data/shapefiles/nigeria/nga_admbnda_adm0_osgof_20190417.shp"
)
algeria = gpd.read_file(
    "../../data/shapefiles/algeria/dza_admbnda_adm0_unhcr_20200120.shp"
)
chad = gpd.read_file("../../data/shapefiles/chad/tcd_admbnda_adm0_ocha.shp")
mali = gpd.read_file("../../data/shapefiles/mali/mli_admbnda_adm0_1m_gov_20211220.shp")
burkinafaso = gpd.read_file(
    "../../data/shapefiles/burkinafaso/bfa_admbnda_adm0_igb_20200323.shp"
)

In [160]:
neighbors = pd.concat([libya, mali, chad, algeria, nigeria, burkinafaso])
countries_of_interest = pd.concat([niger_adm0, mali, burkinafaso])

In [161]:
niger = colocation_ddf[colocation_ddf["country"].isin(["NER"])]

In [162]:
niger_gadm["centroid"] = niger_gadm["geometry"].apply(lambda x: x.centroid)

In [163]:
niger["ds"] = dd.to_datetime(niger["ds"])

In [164]:
niger["month"] = niger["ds"].dt.to_period("M")

In [165]:
def biweekly_grouper(date):
    return (date.year, date.week // 2)

In [166]:
def process_colocation(niger_monthly, time_column):
    niger_monthly = niger_monthly.merge(
        niger_gadm[["GID_2", "centroid"]], left_on="polygon1_id", right_on="GID_2"
    )

    niger_monthly.rename(columns={"centroid": "source_centroid"}, inplace=True)
    niger_monthly = niger_monthly.merge(
        niger_gadm[["GID_2", "centroid"]], left_on="polygon2_id", right_on="GID_2"
    )
    niger_monthly.rename(columns={"centroid": "target_centroid"}, inplace=True)

    niger_monthly.drop(columns=["GID_2_x", "GID_2_y"], inplace=True)
    niger_monthly["source_lat"] = niger_monthly["source_centroid"].apply(lambda p: p.x)
    niger_monthly["source_lon"] = niger_monthly["source_centroid"].apply(lambda p: p.y)
    niger_monthly["target_lat"] = niger_monthly["target_centroid"].apply(lambda p: p.x)
    niger_monthly["target_lon"] = niger_monthly["target_centroid"].apply(lambda p: p.y)

    niger_monthly[time_column] = niger_monthly[time_column].apply(
        lambda x: pd.to_datetime(x)
    )

    return niger_monthly

In [167]:
niger["biweekly"] = niger["ds"].map(biweekly_grouper, meta=("date", "object"))

In [168]:
niger_monthly = niger.groupby(
    [
        "month",
        "polygon1_id",
        "polygon2_id",
        "polygon1_name",
        "polygon2_name",
        "country",
        "polygon_level",
        "is_home_tile_colocation",
    ]
)[["weekly_measured_colocation_rate", "weekly_colocation_rate"]].mean()

In [32]:
niger_monthly = niger_monthly.compute()

In [115]:
# niger_monthly.reset_index().to_csv('../../data/movement/meta/niger_monthly.csv')
niger_monthly = pd.read_csv("../../data/movement/meta/niger_monthly.csv")
niger_monthly.drop(columns="Unnamed: 0", inplace=True)

In [125]:
niger_biweekly = (
    niger.groupby(
        [
            "biweekly",
            "polygon1_id",
            "polygon2_id",
            "polygon1_name",
            "polygon2_name",
            "country",
            "polygon_level",
            "is_home_tile_colocation",
        ]
    )[["weekly_measured_colocation_rate", "weekly_colocation_rate"]]
    .mean()
    .reset_index()
)

In [59]:
niger_biweekly = niger_biweekly.compute()

In [169]:
niger = niger.compute()

In [129]:
niger["ds"] = niger["ds"].apply(lambda x: pd.to_datetime(x))

In [133]:
niger.to_csv("../../data/movement/meta/niger_colocation_weekly.csv")

In [132]:
niger = process_colocation(niger, time_column="ds")

In [148]:
biweekly_date_mapper = (
    niger[["biweekly", "ds"]]
    .drop_duplicates()
    .groupby(["biweekly"])
    .min()
    .to_dict()["ds"]
)
niger_biweekly["biweekly_datetime"] = niger_biweekly["biweekly"].apply(
    lambda x: biweekly_date_mapper[x]
)
niger_biweekly = process_colocation(niger_biweekly, time_column="biweekly_datetime")

In [152]:
# niger_monthly.to_csv('../../data/movement/meta/colocation_niger_monthly.csv')
niger_biweekly.to_csv("../../data/movement/meta/colocation_niger_biweekly.csv")

#### How to use the map?

Click on the FourSquare logo on the top right corner to open the map on full screen. The timeline option at the bottom can be modified to look at movement at different temperal aggregations. 

The base aggregation of the map is at a biweekly level. The average colocation rate represents the rate at which people from the two admin regions interact. 

<iframe width="100%" height="500px" src="https://studio.foursquare.com/map/public/a9710c29-d74d-43c6-a907-b4f9e8d7050c/embed" frameborder="0" allowfullscreen></iframe>

### Observations

- In April 2023 there was a high rate of colocation between Laba and Taboua regions. 
- Overall, there seems to be higher colocation in early 2023 than in early 2024. 